In [15]:
import matplotlib.pyplot as plt
import json
import pandas as pd
from collections import Counter
from tqdm.auto import tqdm
import random
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import plotly.graph_objects as go

In [42]:
df_iss = pd.read_excel('analyzing_shared_conversation/RQ3/rq3_issue.xlsx')
df_iss = df_iss[df_iss['rationale'] != "Others"]
df_iss

,chat_id,prompt,answer,type,urls,rationale,who,where
0,205,I have a simple JavaScript library that I want...,"ChatGPTTo add tests to your project, you'll wa...",issue,https://github.com/openactive/dataset-utils/is...,Reference to a source of solution,Issue author,Issue description
1,345,"I need a program in PHP, which performs the fo...","ChatGPTSure, here's a basic outline of the PHP...",issue,https://github.com/purpleslurple/PurpleSlurple...,Direct link,Issue author,Issue comment
3,200,"rewrite folloing js code to haxe\n""use strict""...",ChatGPTHere's the equivalent code in Haxe:[COD...,issue,https://github.com/kroitor/asciichart/issues/6...,Reference to a potential solution,Issue collaborator,Issue comment
4,150,Is there a way to write exif data to a jpg usi...,"ChatGPTYes, you can manipulate EXIF data in a ...",issue,https://github.com/moom0o/PhotoSphereStudio/is...,Reference to a potential solution,Issue collaborator,Issue comment
5,522,Browse You are an Odoo implementation expert w...,ChatGPTFinished browsingShow workHide workI ap...,issue,https://github.com/odoo/odoo/issues/126891,Reference to a potential solution,Issue author,Issue description
...,...,...,...,...,...,...,...,...
155,565,Here's a regular expression from PEP 263: ^[ \...,ChatGPTFinished workingShow workThe function r...,issue,https://github.com/simonw/symbex/issues/18#iss...,Illustrate an example,Issue author,Issue comment
156,307,Write me a function that takes as input an ope...,ChatGPTSure! To achieve the transformation you...,issue,https://github.com/NVlabs/Deep_Object_Pose/iss...,Reference to a potential solution,Issue collaborator,Issue comment
157,275,How do I add something to the clipboard in a r...,"ChatGPTIn a React app, you can add something t...",issue,https://github.com/neilenns/vatsim-plan-verifi...,Reference to a source of solution,Issue author,Issue description
158,186,This code does not work as it dies not ignore ...,ChatGPTThe issue seems to be due to the way fn...,issue,https://github.com/mpoon/gpt-repository-loader...,Reference to a potential solution,Issue collaborator,Issue comment


In [43]:
# draw graph for rq3 issue sample
iss_who_where = pd.crosstab( df_iss['who'], df_iss['where'])
iss_who_rationale = pd.crosstab( df_iss['who'], df_iss['rationale'])

In [45]:

# Define function to create Sankey diagram
def create_sankey(data):
    row_labels = list(data.index)
    col_labels = list(data.columns)

    nodes = row_labels + col_labels

    # Create links based on heatmap data
    links = []
    source_totals = {row_label: 0 for row_label in row_labels}
    for i, row_label in enumerate(row_labels):
        for j, col_label in enumerate(col_labels):
            value = data.loc[row_label, col_label]
            links.append({
                'source': row_labels.index(row_label),
                'target': len(row_labels) + col_labels.index(col_label),
                'value': data.loc[row_label, col_label]
            })
            source_totals[row_label] += value

    for link in links:
        source_label = row_labels[link['source']]
        link['percentage'] = (link['value'] / source_totals[source_label]) * 100

    # Create Sankey diagram
    return go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=nodes,
        ),
        link=dict(
            source=[link['source'] for link in links],
            target=[link['target'] for link in links],
            value=[link['value'] for link in links],
            label=[f'({link["percentage"]:.2f}%)' for link in links],
        )
    )


In [46]:
# Create Sankey diagrams for the two sets of data
fig = go.Figure(data=[
    create_sankey(iss_who_where),
])

# Set layout
layer_annotations = [

    dict(
        x=0.6,  # Adjust the x-coordinate based on your layout
        y=-0.2,  # Adjust the y-coordinate based on your layout
        xref="paper",
        yref="paper",
        text="Person",
        showarrow=False,
        font=dict(size=12)
    ),
    dict(
        x=1,  # Adjust the x-coordinate based on your layout
        y=-0.2,  # Adjust the y-coordinate based on your layout
        xref="paper",
        yref="paper",
        text="Location",
        showarrow=False,
        font=dict(size=12)
    ),
]
fig.update_layout( font_size=10, annotations=layer_annotations)

# Show the plot
fig.show()

In [47]:
# Create Sankey diagrams for the two sets of data
fig = go.Figure(data=[
    create_sankey(iss_who_rationale),
])

# Set layout
layer_annotations = [

    dict(
        x=0.6,  # Adjust the x-coordinate based on your layout
        y=-0.2,  # Adjust the y-coordinate based on your layout
        xref="paper",
        yref="paper",
        text="Person",
        showarrow=False,
        font=dict(size=12)
    ),
    dict(
        x=1,  # Adjust the x-coordinate based on your layout
        y=-0.2,  # Adjust the y-coordinate based on your layout
        xref="paper",
        yref="paper",
        text="Rationale",
        showarrow=False,
        font=dict(size=12)
    ),
]
fig.update_layout( font_size=10, annotations=layer_annotations)

# Show the plot
fig.show()

In [29]:
df_pr = pd.read_excel('analyzing_shared_conversation/RQ3/rq3_pr.xlsx')
df_pr = df_pr[df_pr['rationale'] != "Others"]
df_pr

,chat_id,prompt,answer,type,urls,rationale,who,where
0,258,const res = await fetch(\n `${this.base...,"ChatGPTChatGPTSure, to create a URL object and...",pr,https://github.com/osmosis-labs/osmosis-fronte...,Reference to a potential solution,PR author,Code review comment
1,171,I need some place on the page to render the co...,ChatGPTYour existing implementation already ha...,pr,https://github.com/hoshotakamoto/banzukesurfin...,Reference to a source of solution,PR author,PR description
2,143,Write a GitHub Action yml file that blocks the...,ChatGPTTo create a GitHub Actions workflow tha...,pr,https://github.com/faker-js/faker/pull/2405#is...,Reference to a source of solution,PR author,PR comment
3,66,Give me an example of a function dispatch tabl...,"ChatGPTIn Go, you can create a function dispat...",pr,https://github.com/c4dt/dela/pull/4#discussion...,Support a claim,Code reviewer,Code review comment
4,24,Based on this TurboWarp custom extension:\n(fu...,"ChatGPTSure, I can help you create an example ...",pr,https://github.com/TurboWarp/extensions/pull/1...,Support a claim,Code reviewer,PR comment
...,...,...,...,...,...,...,...,...
85,117,/*\n * Copyright (C) 2022 NotEnoughUpdates con...,ChatGPTThe code you provided appears to be a p...,pr,https://github.com/NotEnoughUpdates/NotEnoughU...,Reference to a source of solution,PR author,PR comment
86,55,import re\nimport requests\nfrom typing import...,"ChatGPTSure, I'm here to help. Please go ahead...",pr,https://github.com/metaphorsystems/metaphor-py...,Reference to a source of solution,PR author,PR description
87,108,"In older Fortran codes, one often uses the fol...","ChatGPTYes, you are correct. In older Fortran ...",pr,https://github.com/lfortran/lfortran/pull/2446...,Reference to a potential solution,Code reviewer,PR comment
88,48,Write a good subtitle for my website with the ...,"ChatGPT""Effortlessly Manage Household Chores w...",pr,https://github.com/mlhmz/family-task-tracking/...,Reference to a source of solution,PR author,PR description


In [30]:
# draw graph for rq3 pr sample
pr_who_where = pd.crosstab( df_pr['who'], df_pr['where'])
pr_who_rationale = pd.crosstab( df_pr['who'], df_pr['rationale'])

In [41]:
fig = go.Figure(data=[
    create_sankey(pr_who_where)
])

# Set layout
layer_annotations = [

    dict(
        x=0.6,  # Adjust the x-coordinate based on your layout
        y=-0.2,  # Adjust the y-coordinate based on your layout
        xref="paper",
        yref="paper",
        text="Person",
        showarrow=False,
        font=dict(size=12)
    ),
    dict(
        x=1,  # Adjust the x-coordinate based on your layout
        y=-0.2,  # Adjust the y-coordinate based on your layout
        xref="paper",
        yref="paper",
        text="Locaion",
        showarrow=False,
        font=dict(size=12)
    ),
]
fig.update_layout( font_size=10, annotations=layer_annotations)

# Show the plot
fig.show()

In [39]:
fig = go.Figure(data=[
    create_sankey(pr_who_rationale)
])

# Set layout
layer_annotations = [

    dict(
        x=0.6,  # Adjust the x-coordinate based on your layout
        y=-0.2,  # Adjust the y-coordinate based on your layout
        xref="paper",
        yref="paper",
        text="Person",
        showarrow=False,
        font=dict(size=12)
    ),
    dict(
        x=1,  # Adjust the x-coordinate based on your layout
        y=-0.2,  # Adjust the y-coordinate based on your layout
        xref="paper",
        yref="paper",
        text="Rationale",
        showarrow=False,
        font=dict(size=12)
    ),
]
fig.update_layout( font_size=10, annotations=layer_annotations)

# Show the plot
fig.show()